In [15]:
import numpy as np
import pickle

# G-code generation codes are cloned from https://github.com/tibor-barsi/GcodeGenerator. Tibor Barsi is the author of the code, and was a PhD student at the Ladisk lab of the University of Ljubljana
#  We should be careful with crediting the author of the code, if we ever want to make these code public.
from src.g_code_generation_copy.gcode_generator import G_code_generator
from src.g_code_generation_copy.tool_changer_functions import save_params, load_params, printer_start, load_tool, unload_tool, tool_change, take_photo, play_sound, printer_stop
# from src.additional_functions import *
from src.network import Network_custom, replace_brackets
import os

BYU_UW_root = r"G:\.shortcut-targets-by-id\1k1B8zPb3T8H7y6x0irFZnzzmfQPHMRPx\Illimited Lab Projects\Research Projects\Spiders\BYU-UW"

In [16]:
model_name = "parameter_study"

vertices    = np.array([[-12, 1, 0], [-8, 1, 0], [-4, 1, 0], [0, 1, 0], [4, 1, 0], [8, 1, 0], [12, 1, 0],
                        [-12, -1, 0], [-8, -1, 0], [-4, -1, 0], [0, -1, 0], [4, -1, 0], [8, -1, 0], [12, -1, 0],
                        [-8, 2, 0], [-4, 2, 0], [0, 2, 0], [4, 2, 0], [8, 2, 0],
                        [-8, -2, 0], [-4, -2, 0], [0, -2, 0], [4, -2, 0], [8, -2, 0]])*10  # The coordinates of the vertices in mm
edges       = [[0, 1], [1, 2], [2, 3], [3, 4], [4, 5], [5, 6],
               [7, 8], [8, 9], [9, 10], [10, 11], [11, 12], [12, 13],
                [14, 1], [1, 8],[8, 19], [15,2],[2, 9], [9, 20], [16, 3], [3, 10], [10, 21], [17, 4], [4, 11], [11, 22], [18, 5], [5, 12], [12, 23]]
paths       = [[0, 1, 2, 3, 4, 5], [6, 7, 8, 9, 10, 11], [12, 13, 14], [15, 16, 17],[18, 19, 20], [21, 22, 23], [24, 25, 26]]  # The paths of the edges

fixed = [0, 6, 7, 13, 14,15,16,17,18,19,20,21,22,23]  # The fixed vertices
q = np.ones(len(edges))*0.001
directions = np.ones(len(edges))



In [17]:
net = Network_custom.from_fd(vertices, edges, q, fixed, paths = paths, dir = directions)
net.net_plot(vlabels=True, color=True)

In [18]:
TPU = {'E':130, 'v':0.3897, 'p':1.18e-9, 'A':0.11*0.71, 'name': 'TPU conductive'} # Conductive TPU. Manufacturer Ninjatek Eel

net.set_material(TPU) # By saving the material properties in the network, we can easily see how the network was constructed.

# If all elements will get the same material, you can use the following line
E = [TPU['E']]*len(net.edges)
A = [TPU['A']]*len(net.edges)
l0, l_scalar = net.materialize(E, A)

# Yielding the initial lengths of each element. And the following scalar: min(l0/l1)
net.l0, net.l_scalar

(array([39.92913501, 39.84671923, 39.84313203, 39.84313203, 39.84671923,
        39.92913501, 39.92913501, 39.84671923, 39.84313203, 39.84313203,
        39.84671923, 39.92913501, 12.62065604, 14.70594126, 12.62065604,
        13.16472619, 13.61807338, 13.16472619, 13.25539887, 13.43673937,
        13.25539887, 13.16472619, 13.61807338, 13.16472619, 12.62065604,
        14.70594126, 12.62065604]),
 0.9960672574599663)

In [19]:
# net.initialize_shape_optimizer(function_type = 'no optimization',  method = 'L-BFGS-B', params = None)
net.initialize_shape_optimizer(function_type = 'standard',  method = 'L-BFGS-B')
# net.initialize_shape_optimizer(function_type = 'sigmoid',  method = 'L-BFGS-B', params = {'a': 4, 'b': -1})
net.optimize_vertices()
reference_point = [0,0,0]                         # The network will be scaler relative to this point
net.scale_vertices(reference_point, net.l_scalar) # If you don't provide a scalar, it will use network.l_scalar automatically

#  You can plot the scaled network like this
# net.net_plot(color=True, plot_type='scaled')

# Determine the Radius and Angle of the circle that define the arc of each element
R, th = net.arc_param()
# for the arc length (net.l1 * net.l_scalar) is used unless specified otherwise, for the cord length (net.l0) is used unless specified otherwise
xyz = net.arc_points(n = 100) 

net.net_plot(color=True, plot_type='arcs', elables=True)

np.min(R/net.l0), net.l_scalar

(312.05593134855894, 0.9999998113804411)

In [ ]:
printing_params = load_params(r'DATA/NT_Eel_0.2mm_og.json')
# When no interpolation function is provided, linear interpolation is used. No other interpolation function is implemented (yet?).
# net.jump_at_intersection(intersection_width = 2, intersection_height = 2, interpolation_function=None) # These settings are just for visibility, this would be way to much
# These setting are more appropriate for printing
jh = printing_params['layer_height'] * 3
jw = printing_params['d_nozzle'] * 3

net.jump_at_intersection(intersection_width = jw, intersection_height = printing_params['layer_height'], interpolation_function=None) 
net.net_plot(color=True, plot_type='arcs', elables=True)

In [21]:
printing_params = load_params(r'DATA/NT_Eel_0.2mm_og.json')
start_gcode     = open(r'DATA/start_gcode.gcode', 'r').read()
end_gcode       = open(r'DATA/end_gcode.gcode', 'r').read()

temperature_settings = {'first_layer_bed_temperature': 65, 'first_layer_temperature':205, 'K-factor': 0.20}
g_code = replace_brackets(start_gcode, temperature_settings)

comment = ''

bed_width = 250
bed_height = 210

gen = G_code_generator(printing_params=printing_params)

point0, point1 = [20,0,0.3], [90,0,0.3]
g_code += gen.move_to_point(point0[0:2], point0[2] + printing_params['nozzle_lift'], comment='Move to start point')
g_code += gen.move_to_point(point0[0:2], point0[2], comment='Lower Nozzle')
g_code += gen.unretract()
g_code += gen._print_line(
        point0=point0,
        point1=point1,
        move_to_start=False, # move to start point without extruding
        extrude_factor=printing_params['extrude_factor']*2,
        comment=comment)
g_code += gen.retract()
g_code += gen.wipe(np.pi) # Wipe the nozzle horizontally

for path_i, cor_list in enumerate(net.paths_xyz):
    g_code += '\n;Path (' + str(path_i) + ' ' + str(path_i) + ')\n'
    cor_list   = np.array(cor_list)
    # Move the coordinates to the center of the bed and add the layer height
    cor_list[:,0] += bed_width/2
    cor_list[:,1] += bed_height/2 - 1
    cor_list[:,2] += printing_params['layer_height']
    g_code += '\n'
    # Move the coordinates to the start point
    g_code += gen.move_to_point(cor_list[0][0:2], cor_list[0][2] + printing_params['nozzle_lift'], comment='Move to start point')
    # Lower the nozzle
    g_code += gen.move_to_point(cor_list[0][0:2], cor_list[0][2], comment='Lower Nozzle')
    # Unretract the filament
    g_code += gen.unretract()
    # Print the path
    for point0, point1 in zip(cor_list[:-1], cor_list[1:]):
        g_code += gen._print_line(
            point0=point0,
            point1=point1,
            move_to_start=False, # move to start point without extruding
            extrude_factor = printing_params['extrude_factor'],
            comment=comment)
    # Retract the filament
    g_code += gen.retract()
    # Wipe the nozzle
    g_code += gen.wipe_from_last_points(g_code)    
    # Raise the nozzle
    g_code += gen.move_to_point(point1[0:2], point1[2] + printing_params['nozzle_lift'], speed_factor=0.5, comment='Raise Nozzle')

g_code += end_gcode
with open(os.path.join('DATA', 'generated_gcodes', model_name + "_jw" + str(jw) +
                       "_jh" + str(jh) + 
                       "_ef" + str(printing_params['extrude_factor']) +
                       "_s" + str(int(printing_params['print_feedrate'])) +
                       '.gcode'), "w") as g_code_file:
    g_code_file.write(g_code)
print('G-code generated')

G-code generated
